In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import opendatasets as od

# Competition
https://www.kaggle.com/competitions/big-data-derby-2022/overview

# References
https://www.kaggle.com/code/chemdata/intro-to-time-series-analysis-forecasting

https://www.kaggle.com/code/lichtlab/hierarchical-bayesian-estimation-horse-power

# Import Data

In [2]:
if 'inputs' not in os.listdir('.'):
    od.download('https://www.kaggle.com/competitions/big-data-derby-2022')
    os.rename('big-data-derby-2022/','inputs/')

In [3]:
start = pd.read_csv('inputs/nyra_start_table.csv', header=None)
start.columns = ['track_id','race_date','race_number','program_number','weight_carried','jockey','odds','finishing_position']
start.head()

,track_id,race_date,race_number,program_number,weight_carried,jockey,odds,finishing_position
0,AQU,2019-01-01,1,1,123,Dylan Davis,130,2
1,AQU,2019-01-01,1,2,120,Junior Alvarado,295,3
2,AQU,2019-01-01,1,3,118,Jose Lezcano,180,4
3,AQU,2019-01-01,1,4,123,Jomar Garcia,1280,5
4,AQU,2019-01-01,1,5,118,Manuel Franco,1150,1


In [4]:
print(start.dtypes)

track_id              object
race_date             object
race_number            int64
program_number        object
weight_carried         int64
jockey                object
odds                   int64
finishing_position     int64
dtype: object


### nyra_start_table.csv

* **track_id (str)** - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
* **race_date (str)** - date the race took place. YYYY-MM-DD.
* **race_number (str)** - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
* **program_number (str)** - Program number of the horse in the race passed as 3 characters. Should remain 3 characters as it isn't limited to just numbers. Is essentially the unique identifier of the horse in the race.
* **weight_carried (int)** - An integer of the weight carried by the horse in the race.
* **jockey (str)** - Name of the jockey on the horse in the race. 50 character max.
* **odds (int)** - Odds to win the race passed as an integer. Divide by 100 to derive the odds to 1. Example - 1280 would be 12.8-1.
* **position_at_finish (int)** - An integer of the horse's finishing position. (added to the dataset 9/8/22)


In [5]:
race = pd.read_csv('inputs/nyra_race_table.csv')
race.head()

,track_id,race_date,race_number,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time
0,AQU,2019-01-01,1,650,D,MY,36,AOC,80000,1220
1,AQU,2019-01-01,2,600,D,MY,48,MCL,41000,1250
2,AQU,2019-01-01,3,550,D,MY,54,MCL,35000,121
3,AQU,2019-01-01,4,900,D,MY,101,AOC,80000,150
4,AQU,2019-01-01,5,700,D,MY,60,ALW,64000,220


### nyra_race_table.csv

* **track_id (str)** - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
* **race_date (str)** - date the race took place. YYYY-MM-DD.
* **race_number (str)** - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
* **distance_id (int)** - Distance of the race in furlongs passed as an integer. Example - 600 would be 6 furlongs.
* **course_type (char)** - The course the race was run over passed as one character. M - Hurdle, D - Dirt, O - Outer turf, I - Inner turf, T - turf.
* **track_condition (str)** - The condition of the course the race was run on passed as three characters. YL - Yielding, FM - Firm, SY - Sloppy, GD - Good, FT - Fast, MY - Muddy, SF - Soft.
* **run_up_distance (int)** - Distance in feet of the gate to the start of the race passed as an integer.
* **race_type (str)** - The classification of the race passed as as five characters. STK - Stakes, WCL - Waiver Claiming, WMC - Waiver Maiden Claiming, SST - Starter Stakes, SHP - Starter Handicap, CLM - Claiming, STR - Starter Allowance, AOC - Allowance Optionl Claimer, SOC - Starter Optional Claimer, MCL - Maiden Claiming, ALW - Allowance, MSW - Maiden Special Weight.
* **purse (float)** - Purse in US dollars of the race passed as an money with two decimal places.
* **post_time (str)** - Time of day the race began passed as 5 character. Example - 01220 would be 12:20.

In [6]:
tracking = pd.read_csv('inputs/nyra_tracking_table.csv')
tracking.head()

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762


### nyra_tracking_table.csv

* **track_id (str)** - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
* **race_date (str)** - date the race took place. YYYY-MM-DD.
* **race_number (int)** - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
* **program_number (str)** - Program number of the horse in the race passed as 3 characters. Should remain 3 characters as it isn't limited to just numbers. Is essentially the unique identifier of the horse in the race.
* **trakus_index (int)** - The common collection of point of the lat / long of the horse in the race passed as an integer. From what we can tell, it's collected every 0.25 seconds.
* **latitude (float)** - The latitude of the horse in the race passed as a float.
* **longitude (float)** - The longitude of the horse in the race passed as a float.

In [7]:
full_data = pd.read_csv('inputs/nyra_2019_complete.csv', header=None)
full_data.columns = ['track_id','race_date','race_number','program_number','trakus_index','latitude','longitude','distance_id',
               'course_type','track_condition','run_up_distance','race_type','purse','post_time','weight_carried','jockey','odds','finishing_position']
full_data.head()

/tmp/ipykernel_16141/1614228286.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  full_data = pd.read_csv('inputs/nyra_2019_complete.csv', header=None)


,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time,weight_carried,jockey,odds,finishing_position
0,AQU,2019-01-01,9,6,72,40.672902,-73.827607,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
1,AQU,2019-01-01,9,6,73,40.672946,-73.827587,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
2,AQU,2019-01-01,9,6,74,40.672990,-73.827568,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
3,AQU,2019-01-01,9,6,63,40.672510,-73.827781,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8
4,AQU,2019-01-01,9,6,64,40.672553,-73.827762,600,D,GD,48,CLM,25000.0,420,120,Andre Shivnarine Worrie,2090,8


### nyra_2019_complete.csv - 
This file is the combined 3 files into one table. The keys to join them trakus with race - track_id, race_date, race_number. To join trakus with start - track_id, race_date, race_number, program_number.

* **track_id (str)** - 3 character id for the track the race took place at. AQU -Aqueduct, BEL - Belmont, SAR - Saratoga.
* **race_date (str)** - the race took place. YYYY-MM-DD.
* **race_number (str)** - Number of the race. Passed as 3 characters but can be cast or converted to int for this data set.
* **program_number (str)** -  Program number of the horse in the race passed as 3 characters. Should remain 3 characters as it isn't limited to just numbers. Is essentially the unique identifier of the horse in the race.
* **trakus_index (int)** - The common collection of point of the lat / long of the horse in the race passed as an integer. From what we can tell, it's collected every 0.25 seconds.
* **latitude (float)** - The latitude of the horse in the race passed as a float.
* **longitude (float)** - The longitude of the horse in the race passed as a float.
* **distance_id (int)** - Distance of the race in furlongs passed as an integer. Example - 600 would be 6 furlongs.
* **course_type (char)** - The course the race was run over passed as one character. M - Hurdle, D - Dirt, O - Outer turf, I - Inner turf, T - turf.
* **track_condition (str)** -  The condition of the course the race was run on passed as three characters. YL - Yielding, FM - Firm, SY - Sloppy, GD - Good, FT - Fast, MY - Muddy, SF - Soft.
* **run_up_distance (int)** - Distance in feet of the gate to the start of the race passed as an integer.
* **race_type (str)** - The classification of the race passed as as five characters. STK - Stakes, WCL - Waiver Claiming, WMC - Waiver Maiden Claiming, SST - Starter Stakes, SHP - Starter Handicap, CLM - Claiming, STR - Starter Allowance, AOC - Allowance Optionl Claimer, SOC - Starter Optional Claimer, MCL - Maiden Claiming, ALW - Allowance, MSW - Maiden Special Weight.
* **purse (float)** - Purse in US dollars of the race passed as an money with two decimal places
* **post_time (str)** - Time of day the race began passed as 5 character. Example - 01220 would be 12:20.
* **weight_carried (int)** - An integer of the weight carried by the horse in the race.
* **jockey (str)** - Name of the jockey on the horse in the race. 50 character max.
* **odds (int)** - Odds to win the race passed as an integer. Divide by 100 to derive the odds to 1. Example - 1280 would be 12.8-1.
* **position_at_finish (int)** - An integer of the horse's finishing position. (added to the dataset 9/8/22)


# Add, Sort, and Group Data

In [8]:
# Add time to the dataframe
tracking['time'] = tracking['trakus_index'] * .25

### Group Start and Race Data

In [9]:
s_gs = start.groupby(['race_date','race_number'])
r_gs = race.groupby('race_date')

### Sort and Add Columns to Tracking Data

In [10]:
#tracking = tracking.sort_values(['race_date','race_number','program_number','trakus_index'])

tracking = tracking.sort_values(['race_date','race_number','trakus_index'])

#Last value for velocity will be meaningless for each racer
tracking['lat_vel'] = (tracking['latitude'] - tracking['latitude'].shift(-1)) / .25
tracking['lon_vel'] = (tracking['longitude'] - tracking['longitude'].shift(-1)) / .25
tracking['vel_mag'] = (tracking['lat_vel']**2 + tracking['lon_vel']**2)**(1/2)

#Last two values for acceleration will be meaningless for each racer
tracking['lat_acc'] = (tracking['lat_vel'] - tracking['lat_vel'].shift(-1)) / .25
tracking['lon_acc'] = (tracking['lon_vel'] - tracking['lon_vel'].shift(-1)) / .25
tracking['acc_mag'] = (tracking['lat_acc']**2 + tracking['lon_acc']**2)**(1/2)

In [11]:
tracking.head()

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,time,lat_vel,lon_vel,vel_mag,lat_acc,lon_acc,acc_mag
191041,AQU,2019-01-01,1,5,1,40.669386,-73.829145,0.25,-0.000058,0.000242,0.000248,-0.000186,0.001369,0.001381
867361,AQU,2019-01-01,1,1,1,40.669401,-73.829205,0.25,-0.000011,-0.000101,0.000101,-0.000167,-0.000229,0.000284
1133666,AQU,2019-01-01,1,2,1,40.669404,-73.829180,0.25,0.000031,-0.000043,0.000053,0.000661,-0.000056,0.000663
1212464,AQU,2019-01-01,1,3,1,40.669396,-73.829169,0.25,-0.000135,-0.000029,0.000138,-0.001158,0.000188,0.001173
2014641,AQU,2019-01-01,1,4,1,40.669430,-73.829162,0.25,0.000155,-0.000076,0.000173,0.000849,-0.001265,0.001524


### Group Tracking Data

In [12]:
#create function to figure out drafting values
def distance(group,lat,lon,vx,vy):
    return 0
    
    
    
    

In [13]:
individuals = tracking.groupby(['race_date','race_number','program_number'])
times = tracking.groupby(['race_date','race_number','trakus_index'])